# Basic Langchain using Open AI

## Import OpenAI 
- First install Open AI using `pip install openai`
- install env utilities `pip install python-dotenv`

In [9]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import datetime
# Load local env file
_ = load_dotenv(find_dotenv())
# Read openai api key from local environment
llm_model = 'gpt-3.5-turbo'
temperature=0.2
open_ai = OpenAI(
  organization=os.environ['OPENAI_ORG'],
  api_key=os.environ['OPENAI_API_KEY'],
)

# Input Prompt
# test with a prompt
user_msg = "I have not had french fries in a long time, may we have some please !"
style = "Assume role of a Gen Alpha kid persona talking in an informal and exaggerated persona to his/her friend"
prompt = f"""Rephrase the following text that is delimited by triple backticks in {style}. text: ```{user_msg}```"""


## Open AI without Langchain
- Without langchain following block is how open ai calls are made

In [6]:
# define functiont to complete
def get_completion_gpt(prompt,model=llm_model):
    messages = [{"role":"user","content":prompt}]
    response = open_ai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        stream=True,
    )
    return response

response = get_completion_gpt(prompt)
for chunk in response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

I have not indulged in the delectable delight of French fries for quite some time. Would you kindly accompany me in partaking in this savory treat?

## Open AI with LangChain and Parser
- Install Langchain first 
- Use LangSmith for Observability [Official Document](https://python.langchain.com/docs/tutorials/llm_chain/)
- Signup for LangSmith API key [How To...](https://smith.langchain.com/onboarding?organizationId=a1b11d3c-e3c3-4439-9ebc-a077a7c0de67&step=1)
- Trace via LangSmith [Link](https://smith.langchain.com/o/a1b11d3c-e3c3-4439-9ebc-a077a7c0de67/projects/p/9deedd89-2b39-4469-afaa-fe78f12f0bf2?timeModel=%7B%22duration%22%3A%227d%22%7D&tab=0)

In [10]:
#!pip install -U langchain langchain-openai
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=llm_model)

# import system prompt and user prompt from langchain
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content=f"""Translate the following text into a persona that is {style}"""),
    HumanMessage(content=user_msg)
]

# import langchain parser
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

# parser.invoke(llm.invoke(messages))
# More modern way of invoking the response and parsing is by using |
chain = llm | parser
chain.invoke(messages)

"O-M-G, dude, I am literally craving some french fries like you wouldn't believe! It's been FOREVER since I've had those crispy, salty, golden sticks of deliciousness. Can we pleeease go get some ASAP? My taste buds are screaming for some fry action!"